In [37]:
from dotenv import load_dotenv
import sqlite3
import json
import pandas as pd
from typing import List, Dict, Any
import openai
import os
from tqdm import tqdm
load_dotenv()  # laddar .env-filen

client = openai.OpenAI()

print("Libraries imported successfully!")

Libraries imported successfully!


In [38]:
# Connect to the commits database
conn = sqlite3.connect('commits.db')

# Check the total number of commits
total_commits = conn.execute("SELECT COUNT(*) FROM commits").fetchone()[0]
print(f"Total commits in database: {total_commits}")

# Check a sample commit to understand the structure
sample_commit = conn.execute("SELECT * FROM commits LIMIT 1").fetchone()
print("\nSample commit structure:")
print(f"Repo: {sample_commit[0]}")
print(f"SHA: {sample_commit[1]}")
print(f"Title: {sample_commit[2]}")
print(f"Diffs preview: {sample_commit[3][:200]}...")

# Parse the diffs JSON to see structure
sample_diffs = json.loads(sample_commit[3])
print(f"\nNumber of files changed in sample commit: {len(sample_diffs)}")
if sample_diffs:
    print(f"First diff keys: {list(sample_diffs[0].keys())}")
    if 'diff' in sample_diffs[0] and sample_diffs[0]['diff']:
        print(f"Sample diff content: {sample_diffs[0]['diff'][:300]}...")

Total commits in database: 30

Sample commit structure:
Repo: https://github.com/hm-group/gcp-projectfactory
SHA: 88ffb0214cde5bbd88697444e89bf3fe84ec3bad
Title: Merge pull request #15955 from hm-group/add-workload-identity-to-trendie
Diffs preview: [{"filename": "projects/trendie-d.yaml", "diff": "@@ -65,3 +65,6 @@ project:\n     - herman.lindstrom@hm.com\n     - andreas.bergstrom@hm.com\n     - manjunath.satishnaik@hm.com\n+  workloadIdentity:\...

Number of files changed in sample commit: 1
First diff keys: ['filename', 'diff']
Sample diff content: @@ -65,3 +65,6 @@ project:
     - herman.lindstrom@hm.com
     - andreas.bergstrom@hm.com
     - manjunath.satishnaik@hm.com
+  workloadIdentity:
+    - repo: trendie-d-a07f-vertex-pipelines
+...


In [39]:
def analyze_diff_with_prompt(filename: str, diff_content: str, commit_title: str, repo: str) -> Dict[str, Any]:
    """
    Analyze a diff using OpenAI to determine if it's interesting.
    """
    
    if not diff_content or diff_content.strip() == "":
        return {
            "score": 0,
            "confidence": 1.0,
            "reason": "Empty diff",
            "category": "empty"
        }
    
    # Truncate diff if it's too long to avoid token limits
    max_diff_length = 2000
    truncated_diff = diff_content[:max_diff_length]
    if len(diff_content) > max_diff_length:
        truncated_diff += "\n... (truncated)"
    
    prompt = f"""
You are an EXTREMELY strict senior software engineer. Rate this code diff on a scale of 0-100 where you are VERY AGGRESSIVE and STINGY with high scores.

Repository: {repo}
Commit Title: {commit_title}
Filename: {filename}

Diff:
```
{truncated_diff}
```

SCORING GUIDELINES (BE RUTHLESS):
- 90-100: Only for EXCEPTIONAL changes like major security fixes, critical performance breakthroughs, or groundbreaking architectural improvements
- 80-89: Significant security vulnerabilities fixed, major performance optimizations, important API changes
- 70-79: Notable bug fixes, meaningful feature additions, important refactoring
- 60-69: Minor bug fixes, small features, routine improvements
- 40-59: Configuration changes, dependency updates, minor refactoring
- 20-39: Test additions, documentation improvements, code style changes
- 0-19: Trivial changes, formatting, comments, variable renames

BE EXTREMELY STRICT. Most commits should score 20-40. Only truly exceptional changes deserve 80+.

Respond with JSON containing:
- "score": integer 0-100 (BE VERY STINGY WITH HIGH SCORES)
- "confidence": float 0.0-1.0 (how confident you are)
- "category": string (one of: "security", "bugfix", "performance", "database", "configuration", "feature", "refactor", "test", "documentation", "infrastructure", "empty", "general")
- "reason": string (brief explanation of the score)

Respond only with valid JSON.
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an EXTREMELY strict senior software engineer who is very stingy with high scores. Most changes are routine and should score 20-40. Only exceptional changes deserve 70+."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=200
        )
        
        response_text = response.choices[0].message.content.strip()
        
        # Try to parse the JSON response
        try:
            result = json.loads(response_text)
            
            # Validate required fields
            required_fields = ["score", "confidence", "category", "reason"]
            if not all(field in result for field in required_fields):
                raise ValueError("Missing required fields in response")
            
            # Ensure score is between 0 and 100
            result["score"] = max(0, min(100, int(result["score"])))
            
            # Ensure confidence is between 0 and 1
            result["confidence"] = max(0.0, min(1.0, float(result["confidence"])))
            
            # Add metadata
            result["file_type"] = filename.split('.')[-1] if '.' in filename else "unknown"
            result["lines_added"] = truncated_diff.count('\n+')
            result["lines_removed"] = truncated_diff.count('\n-')
            result["total_changes"] = result["lines_added"] + result["lines_removed"]
            
            return result
            
        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error parsing LLM response for {filename}: {e}")
            print(f"Response was: {response_text}")
            
            # Fallback to basic analysis
            return {
                "score": min(30, len(truncated_diff) // 50),  # Very conservative fallback scoring
                "confidence": 0.3,
                "reason": f"LLM parsing failed: {str(e)}",
                "category": "general",
                "file_type": filename.split('.')[-1] if '.' in filename else "unknown",
                "lines_added": truncated_diff.count('\n+'),
                "lines_removed": truncated_diff.count('\n-'),
                "total_changes": truncated_diff.count('\n+') + truncated_diff.count('\n-')
            }
            
    except Exception as e:
        print(f"Error calling OpenAI API for {filename}: {e}")
        
        # Fallback analysis
        return {
            "score": 0,
            "confidence": 0.1,
            "reason": f"API call failed: {str(e)}",
            "category": "general",
            "file_type": filename.split('.')[-1] if '.' in filename else "unknown",
            "lines_added": 0,
            "lines_removed": 0,
            "total_changes": 0
        }

print("LLM-powered diff analysis function defined!")

LLM-powered diff analysis function defined!


In [40]:
def process_all_commits(limit=None):
    """
    Process all commits and analyze each diff.
    """
    query = "SELECT repo, sha, title, diffs FROM commits"
    if limit:
        query += f" LIMIT {limit}"
    
    cursor = conn.execute(query)
    results = []
    
    for row in tqdm(cursor.fetchall(), desc="Processing commits"):
        repo, sha, title, diffs_json = row
        
        try:
            diffs = json.loads(diffs_json)
        except json.JSONDecodeError:
            print(f"Error parsing JSON for commit {sha}")
            continue
        
        # Analyze each file diff in the commit
        for diff_data in diffs:
            filename = diff_data.get('filename', 'unknown')
            diff_content = diff_data.get('diff', '')
            
            analysis = analyze_diff_with_prompt(filename, diff_content, title, repo)
            
            result = {
                'repo': repo,
                'sha': sha,
                'commit_title': title,
                'filename': filename,
                'diff_length': len(diff_content) if diff_content else 0,
                **analysis  # Unpack all analysis results
            }
            
            results.append(result)
    
    return pd.DataFrame(results)

# Process a small sample first to test
print("Processing first 3 commits as a test...")
sample_df = process_all_commits(limit=30)
print(f"Processed {len(sample_df)} file diffs from 3 commits")
print(f"Score distribution:")
print(f"  High scores (70+): {(sample_df['score'] >= 70).sum()}")
print(f"  Medium scores (40-69): {((sample_df['score'] >= 40) & (sample_df['score'] < 70)).sum()}")
print(f"  Low scores (0-39): {(sample_df['score'] < 40).sum()}")
print(f"Average score: {sample_df['score'].mean():.1f}")
sample_df.head()

Processing first 3 commits as a test...


Processing commits: 100%|██████████| 30/30 [02:43<00:00,  5.44s/it]

Processed 57 file diffs from 3 commits
Score distribution:
  High scores (70+): 0
  Medium scores (40-69): 2
  Low scores (0-39): 55
Average score: 33.7


,repo,sha,commit_title,filename,diff_length,score,confidence,category,reason,file_type,lines_added,lines_removed,total_changes
0,https://github.com/hm-group/gcp-projectfactory,88ffb0214cde5bbd88697444e89bf3fe84ec3bad,Merge pull request #15955 from hm-group/add-wo...,projects/trendie-d.yaml,192,35,0.9,configuration,"The change is a simple configuration update, a...",yaml,3,0,3
1,https://github.com/hm-group/gcp-projectfactory,b1cc2b81cd112bf50632dc3a71ef61082a1149d6,Merge pull request #16004 from hm-group/projec...,projects/ml-sa-exploration.yaml,384,35,0.9,configuration,This commit is a simple configuration addition...,yaml,17,0,17
2,https://github.com/hm-group/gcp-projectfactory,adbe6673d057942e4f3e135140d4a8e074c10cfb,Adds requested project,projects/ml-sa-exploration.yaml,384,35,0.9,configuration,This commit simply adds a new project configur...,yaml,17,0,17
3,https://github.com/hm-group/gcp-projectfactory,45b80e834853d14114d05a4439206eded65c8b97,Merge pull request #16003 from hm-group/projec...,projects/sandbox-abhay-0825-1.yaml,373,35,0.9,configuration,This is a simple configuration change for a pr...,yaml,16,0,16
4,https://github.com/hm-group/gcp-projectfactory,1ecc9c06e01d400d30015d23a83bd6fb936e98f8,Adds requested project,projects/sandbox-abhay-0825-1.yaml,373,35,0.9,configuration,This commit simply adds a new project configur...,yaml,16,0,16


In [41]:
sample_df

,repo,sha,commit_title,filename,diff_length,score,confidence,category,reason,file_type,lines_added,lines_removed,total_changes
0,https://github.com/hm-group/gcp-projectfactory,88ffb0214cde5bbd88697444e89bf3fe84ec3bad,Merge pull request #15955 from hm-group/add-wo...,projects/trendie-d.yaml,192,35,0.90,configuration,"The change is a simple configuration update, a...",yaml,3,0,3
1,https://github.com/hm-group/gcp-projectfactory,b1cc2b81cd112bf50632dc3a71ef61082a1149d6,Merge pull request #16004 from hm-group/projec...,projects/ml-sa-exploration.yaml,384,35,0.90,configuration,This commit is a simple configuration addition...,yaml,17,0,17
2,https://github.com/hm-group/gcp-projectfactory,adbe6673d057942e4f3e135140d4a8e074c10cfb,Adds requested project,projects/ml-sa-exploration.yaml,384,35,0.90,configuration,This commit simply adds a new project configur...,yaml,17,0,17
3,https://github.com/hm-group/gcp-projectfactory,45b80e834853d14114d05a4439206eded65c8b97,Merge pull request #16003 from hm-group/projec...,projects/sandbox-abhay-0825-1.yaml,373,35,0.90,configuration,This is a simple configuration change for a pr...,yaml,16,0,16
4,https://github.com/hm-group/gcp-projectfactory,1ecc9c06e01d400d30015d23a83bd6fb936e98f8,Adds requested project,projects/sandbox-abhay-0825-1.yaml,373,35,0.90,configuration,This commit simply adds a new project configur...,yaml,16,0,16
5,https://github.com/hm-group/gcp-projectfactory,02efe890ebd81c1c604f26f3e8cbdeda9b3634f7,Merge pull request #16002 from hm-group/projec...,projects/salesstore-sdl-d.yaml,2171,35,0.90,configuration,This commit is a configuration change for a pr...,yaml,67,0,67
6,https://github.com/hm-group/gcp-projectfactory,839c9c7cbbaef6eb870adc0055d944075e93b1bc,Adds requested project,projects/salesstore-sdl-d.yaml,2171,35,0.90,configuration,This commit is a routine configuration change ...,yaml,67,0,67
7,https://github.com/hm-group/gcp-projectfactory,eb1e70c1e3e5afa9f91678808b6b9b6d7247f2a9,Merge pull request #16001 from hm-group/projec...,projects/audience-mgmt-p.yaml,253,35,1.00,configuration,"This is a simple configuration change, adjusti...",yaml,1,1,2
8,https://github.com/hm-group/gcp-projectfactory,a3eab1901ec2e93ec2546c7711f92fab243c9a8a,Updates project file,projects/audience-mgmt-p.yaml,253,35,0.90,configuration,"This change is a simple configuration update, ...",yaml,1,1,2
9,https://github.com/hm-group/gcp-projectfactory,b8c20b006f44f7ffa3f8878d3925176575dd5d3f,Merge pull request #16000 from hm-group/projec...,projects/cp-eng-prod.yaml,344,35,0.90,configuration,The changes made in this commit are mostly con...,yaml,5,2,7


In [42]:
# First, let's check the current structure of the commits table
cursor = conn.cursor()
table_info = cursor.execute("PRAGMA table_info(commits)").fetchall()
print("Current commits table structure:")
for column in table_info:
    print(f"  {column[1]} ({column[2]})")

# Add new columns to the existing commits table for the analysis results
def add_analysis_columns():
    """
    Add analysis columns to the existing commits table.
    """
    cursor = conn.cursor()
    
    # List of new columns to add
    new_columns = [
        "ADD COLUMN filename TEXT",
        "ADD COLUMN diff_length INTEGER",
        "ADD COLUMN score INTEGER",
        "ADD COLUMN confidence REAL",
        "ADD COLUMN reason TEXT", 
        "ADD COLUMN category TEXT",
        "ADD COLUMN file_type TEXT",
        "ADD COLUMN lines_added INTEGER",
        "ADD COLUMN lines_removed INTEGER",
        "ADD COLUMN total_changes INTEGER"
    ]
    
    # Add each column (SQLite will ignore if column already exists)
    for column in new_columns:
        try:
            cursor.execute(f"ALTER TABLE commits {column}")
            print(f"Added column: {column.split()[-2]}")
        except sqlite3.OperationalError as e:
            if "duplicate column name" in str(e):
                print(f"Column {column.split()[-2]} already exists")
            else:
                print(f"Error adding column {column.split()[-2]}: {e}")
    
    conn.commit()
    print("Finished adding analysis columns to commits table!")

# Function to update the commits table with analysis data
def update_commits_with_analysis(df):
    """
    Update the commits table with analysis data from the dataframe.
    Since we need to handle multiple files per commit, we'll need to create
    separate rows for each file or aggregate the data.
    """
    cursor = conn.cursor()
    
    # Group by commit (sha) and aggregate the analysis data
    commit_aggregates = df.groupby('sha').agg({
        'score': 'mean',  # Average score across all files
        'confidence': 'mean',  # Average confidence
        'category': lambda x: ', '.join(x.unique()),  # Concatenate unique categories
        'file_type': lambda x: ', '.join(x.unique()),  # Concatenate unique file types
        'lines_added': 'sum',  # Total lines added
        'lines_removed': 'sum',  # Total lines removed
        'total_changes': 'sum',  # Total changes
        'diff_length': 'sum',  # Total diff length
        'reason': lambda x: x.iloc[0]  # Take first reason as representative
    }).round(2)
    
    # Update each commit with aggregated analysis data
    for sha, data in commit_aggregates.iterrows():
        cursor.execute('''
            UPDATE commits 
            SET score = ?, confidence = ?, reason = ?, category = ?, 
                file_type = ?, lines_added = ?, lines_removed = ?, 
                total_changes = ?, diff_length = ?
            WHERE sha = ?
        ''', (
            data['score'],
            data['confidence'], 
            data['reason'],
            data['category'],
            data['file_type'],
            data['lines_added'],
            data['lines_removed'],
            data['total_changes'],
            data['diff_length'],
            sha
        ))
    
    conn.commit()
    print(f"Updated {len(commit_aggregates)} commits with analysis data!")

# Execute the functions
add_analysis_columns()
update_commits_with_analysis(sample_df)

# Verify the updates
verification_query = conn.execute("""
    SELECT repo, sha, title, score, category, lines_added, lines_removed 
    FROM commits 
    WHERE score IS NOT NULL 
    LIMIT 5
""").fetchall()

print(f"\nSample of updated commits:")
for row in verification_query:
    print(f"Repo: {row[0]}, SHA: {row[1][:8]}..., Score: {row[3]}, Category: {row[4]}")

Current commits table structure:
  repo (TEXT)
  sha (TEXT)
  title (TEXT)
  diffs (TEXT)
  filename (TEXT)
  diff_length (INTEGER)
  score (INTEGER)
  confidence (REAL)
  reason (TEXT)
  category (TEXT)
  file_type (TEXT)
  lines_added (INTEGER)
  lines_removed (INTEGER)
  total_changes (INTEGER)
Column filename already exists
Column diff_length already exists
Column score already exists
Column confidence already exists
Column reason already exists
Column category already exists
Column file_type already exists
Column lines_added already exists
Column lines_removed already exists
Column total_changes already exists
Finished adding analysis columns to commits table!
Updated 30 commits with analysis data!

Sample of updated commits:
Repo: https://github.com/hm-group/gcp-projectfactory, SHA: 88ffb021..., Score: 35, Category: configuration
Repo: https://github.com/hm-group/gcp-projectfactory, SHA: b1cc2b81..., Score: 35, Category: configuration
Repo: https://github.com/hm-group/gcp-project